In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, PowerTransformer,StandardScaler,MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report ,confusion_matrix

import pickle
import warnings
warnings.filterwarnings(action='ignore')



In [7]:
df = pd.read_csv('outlier_removed.csv')
df.drop(columns=['Unnamed: 0'],inplace=True)
df.head()

,age,fnlwgt,education,educational-num,marital,relationship,race,sex,capital,capital.1,hours,country,salary,working,occupation
0,25.0,226802.0,11th,7.0,Never-married,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K,Private,Machine-op-inspct
1,38.0,89814.0,HS-grad,9.0,Married-civ-spouse,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K,Private,Farming-fishing
2,28.0,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Husband,White,Male,0.0,0.0,40.0,United-States,>50K,Local-gov,Protective-serv
3,44.0,160323.0,Some-college,10.0,Married-civ-spouse,Husband,Black,Male,0.0,0.0,40.0,United-States,>50K,Private,Machine-op-inspct
4,18.0,103497.0,Some-college,10.0,Never-married,Own-child,White,Female,0.0,0.0,32.5,United-States,<=50K,False,False


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              48842 non-null  float64
 1   fnlwgt           48842 non-null  float64
 2   education        48842 non-null  object 
 3   educational-num  48842 non-null  float64
 4   marital          48842 non-null  object 
 5   relationship     48842 non-null  object 
 6   race             48842 non-null  object 
 7   sex              48842 non-null  object 
 8   capital          48842 non-null  float64
 9   capital.1        48842 non-null  float64
 10  hours            48842 non-null  float64
 11  country          48842 non-null  object 
 12  salary           48842 non-null  object 
 13  working          48842 non-null  object 
 14  occupation       48842 non-null  object 
dtypes: float64(6), object(9)
memory usage: 5.6+ MB


In [9]:
df.isnull().sum()

age                0
fnlwgt             0
education          0
educational-num    0
marital            0
relationship       0
race               0
sex                0
capital            0
capital.1          0
hours              0
country            0
salary             0
working            0
occupation         0
dtype: int64

In [10]:
df.head()

,age,fnlwgt,education,educational-num,marital,relationship,race,sex,capital,capital.1,hours,country,salary,working,occupation
0,25.0,226802.0,11th,7.0,Never-married,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K,Private,Machine-op-inspct
1,38.0,89814.0,HS-grad,9.0,Married-civ-spouse,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K,Private,Farming-fishing
2,28.0,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Husband,White,Male,0.0,0.0,40.0,United-States,>50K,Local-gov,Protective-serv
3,44.0,160323.0,Some-college,10.0,Married-civ-spouse,Husband,Black,Male,0.0,0.0,40.0,United-States,>50K,Private,Machine-op-inspct
4,18.0,103497.0,Some-college,10.0,Never-married,Own-child,White,Female,0.0,0.0,32.5,United-States,<=50K,False,False


In [11]:
df1 = df.copy()

In [12]:
df1.shape

(48842, 15)

In [13]:
x1 = df1.drop(columns=['salary'],axis=1)
y = df1['salary']

In [15]:
x1 = x1.select_dtypes(include=[object])
x1.head()

,education,marital,relationship,race,sex,country,working,occupation
0,11th,Never-married,Own-child,Black,Male,United-States,Private,Machine-op-inspct
1,HS-grad,Married-civ-spouse,Husband,White,Male,United-States,Private,Farming-fishing
2,Assoc-acdm,Married-civ-spouse,Husband,White,Male,United-States,Local-gov,Protective-serv
3,Some-college,Married-civ-spouse,Husband,Black,Male,United-States,Private,Machine-op-inspct
4,Some-college,Never-married,Own-child,White,Female,United-States,False,False


In [16]:
x1.columns

Index(['education', 'marital', 'relationship', 'race', 'sex', 'country',
       'working', 'occupation'],
      dtype='object')

In [17]:
from sklearn import preprocessing

In [19]:
le = preprocessing.LabelEncoder()

In [20]:
df1 = x1.apply(le.fit_transform)

In [21]:
df1.head()

,education,marital,relationship,race,sex,country,working,occupation
0,1,4,3,2,1,39,4,7
1,11,2,0,4,1,39,4,5
2,7,2,0,4,1,39,2,11
3,15,2,0,2,1,39,4,7
4,15,4,3,4,0,39,0,4


In [22]:
df1.shape

(48842, 8)

In [30]:
df2 = df.select_dtypes(include='number')
df2.head()

,age,fnlwgt,educational-num,capital,capital.1,hours
0,25.0,226802.0,7.0,0.0,0.0,40.0
1,38.0,89814.0,9.0,0.0,0.0,50.0
2,28.0,336951.0,12.0,0.0,0.0,40.0
3,44.0,160323.0,10.0,0.0,0.0,40.0
4,18.0,103497.0,10.0,0.0,0.0,32.5


In [31]:
df2.shape

(48842, 6)

In [32]:
result = pd.concat([df1,df2],axis=1)

In [33]:
result


,education,marital,relationship,race,sex,country,working,occupation,age,fnlwgt,educational-num,capital,capital.1,hours
0,1,4,3,2,1,39,4,7,25.0,226802.0,7.0,0.0,0.0,40.0
1,11,2,0,4,1,39,4,5,38.0,89814.0,9.0,0.0,0.0,50.0
2,7,2,0,4,1,39,2,11,28.0,336951.0,12.0,0.0,0.0,40.0
3,15,2,0,2,1,39,4,7,44.0,160323.0,10.0,0.0,0.0,40.0
4,15,4,3,4,0,39,0,4,18.0,103497.0,10.0,0.0,0.0,32.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,7,2,5,4,0,39,4,13,27.0,257302.0,12.0,0.0,0.0,38.0
48838,11,2,0,4,1,39,4,7,40.0,154374.0,9.0,0.0,0.0,40.0
48839,11,6,4,4,0,39,4,0,58.0,151910.0,9.0,0.0,0.0,40.0
48840,11,4,3,4,1,39,4,0,22.0,201490.0,9.0,0.0,0.0,32.5


In [35]:
df_2 = pd.get_dummies(y,drop_first=True)

In [36]:
df_2

,>50K
0,0
1,0
2,1
3,1
4,0
...,...
48837,0
48838,1
48839,0
48840,0


In [37]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(result,df_2)

In [39]:
#logistic regression
lr = LogisticRegression()
pipe = Pipeline([
    ('lr', lr)
    ])
pipe.fit(x_train,y_train)
y_pred =pipe.predict(x_test)
accuracy_score(y_test,y_pred)

0.7595610515109328

In [40]:
#confusion matrix
confusion_mat = confusion_matrix(y_test,y_pred)
print("Confusion matrix: \n",confusion_mat)
print(classification_report(y_test,y_pred))

Confusion matrix: 
 [[9275    0]
 [2936    0]]
              precision    recall  f1-score   support

           0       0.76      1.00      0.86      9275
           1       0.00      0.00      0.00      2936

    accuracy                           0.76     12211
   macro avg       0.38      0.50      0.43     12211
weighted avg       0.58      0.76      0.66     12211



In [42]:
## using svc
svc = SVC()
pipe = Pipeline([
    ('svc', svc)
    ])
pipe.fit(x_train,y_train)
y_pred =pipe.predict(x_test)
accuracy_score(y_test,y_pred)


0.7595610515109328

In [43]:
#confusion matrix
confusion_mat = confusion_matrix(y_test,y_pred)
print("Confusion matrix: \n",confusion_mat)
print(classification_report(y_test,y_pred))

Confusion matrix: 
 [[9275    0]
 [2936    0]]
              precision    recall  f1-score   support

           0       0.76      1.00      0.86      9275
           1       0.00      0.00      0.00      2936

    accuracy                           0.76     12211
   macro avg       0.38      0.50      0.43     12211
weighted avg       0.58      0.76      0.66     12211



In [44]:
#using gaussiannb
gaussian = GaussianNB()
pipe = Pipeline([
    ('nb', gaussian)
    ])
pipe.fit(x_train,y_train)
y_pred =pipe.predict(x_test)
accuracy_score(y_test,y_pred)


0.7882237327000245

In [45]:
#confusion matrix
confusion_mat = confusion_matrix(y_test,y_pred)
print("Confusion matrix: \n",confusion_mat)
print(classification_report(y_test,y_pred))

Confusion matrix: 
 [[8582  693]
 [1893 1043]]
              precision    recall  f1-score   support

           0       0.82      0.93      0.87      9275
           1       0.60      0.36      0.45      2936

    accuracy                           0.79     12211
   macro avg       0.71      0.64      0.66     12211
weighted avg       0.77      0.79      0.77     12211



In [46]:
#using KNN neighbour
knn = KNeighborsClassifier()
pipe = Pipeline([
    ('knn', knn)
    ])
pipe.fit(x_train,y_train)
y_pred =pipe.predict(x_test)
accuracy_score(y_test,y_pred)

0.7220538858406355

In [47]:
#Confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion matrix: \n",confusion_mat)
print(classification_report(y_test, y_pred))

Confusion matrix: 
 [[8172 1103]
 [2291  645]]
              precision    recall  f1-score   support

           0       0.78      0.88      0.83      9275
           1       0.37      0.22      0.28      2936

    accuracy                           0.72     12211
   macro avg       0.58      0.55      0.55     12211
weighted avg       0.68      0.72      0.70     12211



In [48]:
#using decision tree
tree = DecisionTreeClassifier()
pipe = Pipeline([
    ('tree', tree)
    ])
pipe.fit(x_train,y_train)
y_pred =pipe.predict(x_test)
accuracy_score(y_test,y_pred)


0.772827778232741

In [49]:
#Confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion matrix: \n",confusion_mat)
print(classification_report(y_test, y_pred))

Confusion matrix: 
 [[7850 1425]
 [1349 1587]]
              precision    recall  f1-score   support

           0       0.85      0.85      0.85      9275
           1       0.53      0.54      0.53      2936

    accuracy                           0.77     12211
   macro avg       0.69      0.69      0.69     12211
weighted avg       0.77      0.77      0.77     12211



In [51]:
#using random forest
rf = RandomForestClassifier()
pipe = Pipeline([
    ('rf',rf)
    ])
pipe.fit(x_train.values, y_train.values)
y_pred = pipe.predict(x_test.values)
accuracy_score(y_test, y_pred)

0.8234378838751945

In [52]:
#Confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion matrix: \n",confusion_mat)
print(classification_report(y_test, y_pred))

Confusion matrix: 
 [[8433  842]
 [1314 1622]]
              precision    recall  f1-score   support

           0       0.87      0.91      0.89      9275
           1       0.66      0.55      0.60      2936

    accuracy                           0.82     12211
   macro avg       0.76      0.73      0.74     12211
weighted avg       0.82      0.82      0.82     12211

